# pyESM WORKFLOW

CASE A 

Generation of a new model from scratch, including database and data input

(settings -> use_existing_database: False)

In [ ]:
# Create an instance of Model class based on the model settings.
# This generates the model's set input file/s to be filled 
# and the blank SQL database.
from src.frontend.model import Model

test = Model(
    log_level='debug',
    log_format='minimal',
)

In [ ]:
# load model sets from the xlsx files filled by the user
# Note: data validation in excel according to the constant file is envisaged
test.load_model_coordinates()

In [ ]:
# Once sets has compiled in excel and loaded in the database, 
# blank database is generated.
# Note: an error is raised at the first launch of the method, 
# disappearing the second run.
test.generate_blank_database()

In [ ]:
# Load input file/s data into the sql database.
# Can be used to update data in the sql database without generating a new 
# Model instance
test.load_data_files_to_database(operation='overwrite')

In [ ]:
# Initialize cvxpy problem.and
# Fetch data from sql database to cvxpy exogenous variables,
# (to be done: generate equations and objective function)
test.initialize_problem()

CASE B

Generation of a new model working with existing database and data input files.

(settings -> use_existing_database: True)

In [1]:
# Generating a Model instance referring to existing model folder and input data.
from src.frontend.model import Model

test = Model(
    log_level='debug',
    log_format='minimal',
)

test.load_model_coordinates()
test.initialize_problem()

INFO | Model | 'Model' object initialization...
INFO | Model.file_manager | 'FileManager' object generated.
INFO | Model | 'Model' object: loading settings.
DEBUG | Model.file_manager | File 'settings.yml' loaded.
INFO | Model | 'Model' object: loading paths from settings.
INFO | Model.core | 'Core' object initialization...
INFO | Model.core | Skipping model directory generation.
INFO | Model.core.sql_manager | 'SQLManager' object Generated.
INFO | Model.core.index | 'Index' object initialization...
DEBUG | Model.core.index | Loading variables table headers and label headers to Index.
INFO | Model.core.index | 'Index' object initialized.
INFO | Model.core.database | 'Database' object initialization...
INFO | Model.core.database | 'Database' object initialized.
INFO | Model.core.problem | 'Problem' object initialization...
INFO | Model.core.problem | 'Problem' object initialized.
INFO | Model.core | 'Core' initialized.
INFO | Model.pbi_manager | 'PBIManager' object generated.
INFO | Mod

In [ ]:
# In case it is needed, update data without re-generate a new instance of Model class.
test.update_database_and_problem()

In [ ]:
test.load_results_to_database(operation='update')

TESTS ---

In [2]:
test.core.index.sets.technologies


symbol: t
table_name: _set_TECHNOLOGIES
table_headers: {'id': ['t_ID', 'INTEGER PRIMARY KEY'], 'name': ['t_Name', 'TEXT'], 'category': ['t_Category', 'TEXT']}
set_categories: {'t.s': 'Supply technology', 't.d': 'Demand technology'}

In [4]:
test.core.index.variables.X


symbol: X
name: total technology activity vector
type: endogenous
set_headers_key: name
coordinates_info: {'scenarios': None, 'datetime': None, 'technologies': {'set_categories': 't.s'}}
shape: ['technologies', 1]
variable_fields: {'scenarios': ['s_Name', 'TEXT'], 'datetime': ['dt_Name', 'TEXT'], 'technologies': ['t_Name', 'TEXT']}
table_headers: {'id': ['id', 'INTEGER PRIMARY KEY'], 'scenarios': ['s_Name', 'TEXT'], 'datetime': ['dt_Name', 'TEXT'], 'technologies': ['t_Name', 'TEXT']}
coordinates: {'s_Name': ['Reference'], 'dt_Name': ['2023', '2024'], 't_Name': ['Gas power plant', 'Steel factory', 'PV power plant']}
foreign_keys: {'s_Name': ('s_Name', '_set_SCENARIOS'), 'dt_Name': ('dt_Name', '_set_DATETIME'), 't_Name': ('t_Name', '_set_TECHNOLOGIES')}
sets_parsing_hierarchy: {'scenarios': 's_Name', 'datetime': 'dt_Name'}

In [ ]:
import numpy as np
values = np.ones((3,1))

variable.data['variable'][0].value = values

In [ ]:
import pandas as pd
from src.util import constants
from src.util import util

field_id = constants._STD_ID_FIELD['id'][0]

col_to_update = constants._STD_VALUES_FIELD['values'][0]
cols_common = list(variable.coordinates.keys())

new_df = variable.reshaping_variable_data(0)

new_values = {
    's_Name': 'Reference',
    'dt_Name': '2023',
    't_Name': 'Final demand',
    'values': 15.0
}

new_df.loc[len(new_df)] = new_values
new_df.at[0,'values'] = 2

test.core.sqltools.open_connection()
existing_df = test.core.sqltools.table_to_dataframe(table_name=variable.symbol)
existing_df.drop(columns=['id'],inplace=True)
test.core.sqltools.close_connection()

print('\n')
print('existing_df: \n', existing_df)
print('\n')
print('new_df: \n', new_df)

In [ ]:
variable=test.core.index.variables.u

test.core.database.sqltools.open_connection()
raw_data = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name=variable.symbol,
    filters_dict=variable.data['filter'][0]
)
test.core.database.sqltools.close_connection()

raw_data

In [ ]:
test.core.database.sqltools.open_connection()

var = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name='u',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        'f_Name': ['Energy', 'Steel'],
        't_Name': ['PV power plant', 'Steel factory', 'Gas power plant']
    }
)

# filter_1 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='f_Name',
#     parent_table_name='_set_FLOWS',
#     parent_table_fields={
#         'f_Category': ['Product flow'],
#     }
# )

# filter_2 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='dt_Name',
#     parent_table_name='_set_DATETIME',
#     parent_table_fields={
#         'dt_Name': [2023],
#     }
# )

# var_custom_filter = test.core.database.sqltools.filtered_table_to_dataframe(
#     table_name='v',
#     filters_dict={
#         **filter_1,
#         **filter_2,
#         **{'f_Name': ['Steel']},
#     }
# )

test.core.database.sqltools.close_connection()

# filter_1, filter_2
var

Other commands

In [ ]:
# delete all the information included in the model data folder, if necessary
test.erase_model()